In [1]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re
import nltk

c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_excel("./Dataset.xlsx")
df.head()

# change a1 to (int)1, a2 to (int)2, ... so on till a5 to (int)5 for Label column
df['Label'] = df['Label'].str.replace('a', '')
df['Label'] = df['Label'].astype(int)
df.head()

,Sentence,Label
0,The development of reinforcement techniques in...,1
1,SERMO asks the user on a daily basis on events...,2
2,This finding renders the IL-Cl passivated pero...,5
3,"In this paper, an investigation has been carri...",3
4,"In this survey, we perform a broad and thoroug...",3


In [4]:
# model_id = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
'''
different multilabel classification models:
sentence-transformers/paraphrase-MiniLM-L6-v2
sentence-transformers/paraphrase-mpnet-base-v2
sentence-transformers/paraphrase-TinyBERT-L6-v2
sentence-transformers/paraphrase-distilroberta-base-v2
sentence-transformers/paraphrase-xlm-r-multilingual-v1
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
one-vs-rest,multi-output, classifier-chain -> 3 options available for multi-target-strategy
'''
model = SetFitModel.from_pretrained(model_id, multi_target_strategy='one-vs-rest')
# model = SentenceTransformer(model_id)


c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nishi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN thi

In [5]:
# split df into train and test in 4:1 ratio
train_df = df.sample(frac=0.8, random_state=200)
test_df = df.drop(train_df.index).reset_index(drop=True)

In [22]:
# change column names of train and test dataframes from Sentence to text and Label to labels
train_df.columns = ['text', 'labels']
test_df.columns = ['text', 'labels']
# test_df
# reset index of train and test dataframes
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [25]:
# use SetFitTrainer to train the model
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_df,
    eval_dataset=test_df,
    loss_class=CosineSimilarityLoss,
    num_iterations=100,
    batch_size=32,
    # learning_rate=0.00002,
    samples_per_label= 20,
    # warmup_steps=100,
)

In [27]:
trainer.column_mapping = { # maps column names to model inputs
    'text': 'text',
    'labels': 'labels',
}

In [28]:
trainer.train()

AttributeError: 'DataFrame' object has no attribute 'column_names'